In [1]:
import numpy as np
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Model, Reaction, Metabolite
from os.path import join

data_dir = "/Users/phivri/Documents/GitHub/Biosustain/MoGeoModel"

In [2]:
Cljung_Model = cobra.io.read_sbml_model(join(data_dir,'Models/C-ljungdahlii_iHN637.xml'))

medium = Cljung_Model.medium
medium['EX_h2_e'] = 1000
medium['EX_h2o_e'] = 1000
medium['EX_k_e'] = 0
medium['EX_fru_e'] = 0
medium['EX_mg2_e'] = 0
medium['EX_mn2_e'] = 0
medium['EX_na1_e'] = 0
medium['EX_nh4_e'] = 0
medium['EX_ni2_e'] = 0
medium['EX_pi_e'] = 0
medium['EX_mobd_e'] = 0
medium['EX_zn2_e'] = 0
medium['EX_ribflv_e'] = 0
medium['EX_btn_e'] = 0
medium['EX_so4_e'] = 0
medium['EX_thm_e'] = 0
medium['EX_ca2_e'] = 0
medium['EX_cl_e'] = 0
medium['EX_co2_e'] = 10
medium['EX_cobalt2_e'] = 0
medium['EX_cu2_e'] = 0
medium['EX_fe2_e'] = 0
medium['EX_fol_e'] = 0
medium['EX_h_e'] = 1000
Cljung_Model.medium = medium

Using license file /Users/phivri/gurobi.lic
Academic license - for non-commercial use only - expires 2022-07-23


In [3]:
Cljung_Model.reactions.ATPM.bounds = 0, 1000

Cljung_Model.reactions.ACt2r.knock_out()

acT = Reaction("ACt")
acT.name = "Acetate transporter"
acT.add_metabolites({ 
    Cljung_Model.metabolites.ac_c: -1.0,
    Cljung_Model.metabolites.ac_e: 1.0
})

mthfd2 = Reaction("MTHFD2")
mthfd2.name = "Methylenetetrahydrofolate dehydrogenase (NAD)"

mthfd2.add_metabolites({
    Cljung_Model.metabolites.mlthf_c: -1.0,
    Cljung_Model.metabolites.nad_c: -1.0,
    Cljung_Model.metabolites.methf_c: 1.0,
    Cljung_Model.metabolites.nadh_c: 1.0,
})
print(mthfd2.check_mass_balance())    #ensure correct mass balance
print(mthfd2.bounds)                  #Ensure correct bounds
print(mthfd2.reaction)

Cljung_Model.add_reactions([acT, mthfd2])
Cljung_Model.reactions.MTHFD2.bounds = -1000.0, 1000.0


{}
(0.0, 1000.0)
mlthf_c + nad_c --> methf_c + nadh_c


In [4]:
Cljung_Model.reactions.ACACT1r.bounds = 0,1000.0
Cljung_Model.reactions.HACD1.bounds = 0,1000.0
Cljung_Model.reactions.ECOAH1.bounds = 0,1000.0
Cljung_Model.reactions.ACOAD1z.bounds = 0,1000.0
Cljung_Model.reactions.PBUTT.bounds = 0,1000.0
Cljung_Model.reactions.BUTKr.bounds = -1000.0,0
Cljung_Model.reactions.BTCOARx.bounds = 0,1000.0
Cljung_Model.reactions.ALCD4.bounds = -1000.0,0
Cljung_Model.reactions.BTOHt.bounds = -1000.0, 1000.0
Cljung_Model.reactions.BUTt.bounds = -1000.0, 1000.0
Cljung_Model.reactions.HYDFDN.bounds = -1000.0, 1000.0

In [5]:
WLModel = Model('Model Core WL')

WLModel.add_reactions([
#Transport:
Cljung_Model.reactions.EX_h2_e,
Cljung_Model.reactions.EX_h2o_e,
Cljung_Model.reactions.EX_co2_e,
Cljung_Model.reactions.EX_co_e,
Cljung_Model.reactions.EX_pi_e,
Cljung_Model.reactions.EX_h_e,
Cljung_Model.reactions.EX_ac_e,
#Exchange:
Cljung_Model.reactions.H2td,
Cljung_Model.reactions.H2Ot,
Cljung_Model.reactions.CO2t,
Cljung_Model.reactions.COt,
Cljung_Model.reactions.ACt,
#WL:
Cljung_Model.reactions.FDH7,
Cljung_Model.reactions.FTHFLi,
Cljung_Model.reactions.MTHFC,
Cljung_Model.reactions.MTHFD2,
Cljung_Model.reactions.MTHFR5,
Cljung_Model.reactions.METR,
Cljung_Model.reactions.CODH4,
Cljung_Model.reactions.CODH_ACS,
Cljung_Model.reactions.PTAr,
Cljung_Model.reactions.ACKr,
Cljung_Model.reactions.ACAFDOR,
Cljung_Model.reactions.ALCD2x,
#Acetate prod.:
Cljung_Model.reactions.HYDFDN,
Cljung_Model.reactions.RNF,
Cljung_Model.reactions.ATPS4r])
#Sinks:
WLModel.add_boundary(WLModel.metabolites.get_by_id("atp_c"), type="sink")
WLModel.add_boundary(WLModel.metabolites.get_by_id("adp_c"), type="sink")
WLModel.add_boundary(WLModel.metabolites.get_by_id("pi_c"), type="sink")

Reaction identifier,SK_pi_c
Name,Phosphate sink
Memory address,0x07f78896a7128
Stoichiometry,pi_c <=> Phosphate <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [6]:
WLModel.objective = "EX_ac_e"

with WLModel:
    medium = WLModel.medium
    medium['EX_co2_e'] = 0
    medium['EX_co_e'] = 10
    medium['EX_h2_e'] = 0
    WLModel.medium = medium
    print(WLModel.objective, ":", WLModel.slim_optimize(), "\n")
    Acetate_ex_carb = WLModel.slim_optimize()
    CO2_ex_carb = WLModel.reactions.EX_co2_e.flux
    H2_ex_carb = WLModel.reactions.EX_h2_e.flux
    CO_ex_carb = WLModel.reactions.EX_co_e.flux
    H2O_ex_carb = WLModel.reactions.EX_h2o_e.flux

with WLModel:
    medium = WLModel.medium
    medium['EX_co2_e'] = 10
    medium['EX_co_e'] = 0
    medium['EX_h2_e'] = 1000
    WLModel.medium = medium
    print(WLModel.objective, ":", WLModel.slim_optimize(), "\n")
    Acetate_ex_hom = WLModel.slim_optimize()
    CO2_ex_hom = WLModel.reactions.EX_co2_e.flux
    H2_ex_hom = WLModel.reactions.EX_h2_e.flux
    CO_ex_hom = WLModel.reactions.EX_co_e.flux
    H2O_ex_hom = WLModel.reactions.EX_h2o_e.flux
    
print(f"Carboxydotroph: {-CO_ex_carb/Acetate_ex_carb} CO + {round(-H2O_ex_carb/Acetate_ex_carb,2)} H2O => {CO2_ex_carb/Acetate_ex_carb} CO2 + acetate")
print(f"Homoacetogen: {-CO2_ex_hom/Acetate_ex_hom} CO2 + {-H2_ex_hom/Acetate_ex_hom} H2 => {round(H2O_ex_hom/Acetate_ex_hom,2)} H2O + acetate")

Maximize
1.0*EX_ac_e - 1.0*EX_ac_e_reverse_0be96 : 2.5 

Maximize
1.0*EX_ac_e - 1.0*EX_ac_e_reverse_0be96 : 5.0 

Carboxydotroph: 4.0 CO + 0.33 H2O => 2.0 CO2 + acetate
Homoacetogen: 2.0 CO2 + 4.0 H2 => 2.33 H2O + acetate


Multiply by given factor to get an acetate production flux of 10, suited as input for the geobacillus model

In [7]:
Acetate_exc = 10
CO2_exc_hom = CO2_ex_hom/Acetate_ex_hom*Acetate_exc
H2_exc_hom = H2_ex_hom/Acetate_ex_hom*Acetate_exc
CO_exc_hom = CO_ex_hom/Acetate_ex_hom*Acetate_exc
H2O_exc_hom = H2O_ex_hom/Acetate_ex_hom*Acetate_exc

print("CO2_ex_hom", CO2_exc_hom)
print("H2_ex_hom", H2_exc_hom)
print("CO_ex_hom", CO_exc_hom)
print("H2O_ex_hom", H2O_exc_hom)

CO2_exc_carb = CO2_ex_carb/Acetate_ex_carb*Acetate_exc
H2_exc_carb = H2_ex_carb/Acetate_ex_carb*Acetate_exc
CO_exc_carb = CO_ex_carb/Acetate_ex_carb*Acetate_exc
H2O_exc_carb = H2O_ex_carb/Acetate_ex_carb*Acetate_exc

print("CO2_ex_carb", CO2_exc_carb)
print("H2_ex_carb", H2_exc_carb)
print("CO_ex_carb", CO_exc_carb)
print("H2O_ex_carb", H2O_exc_carb)


CO2_ex_hom -20.0
H2_ex_hom -40.0
CO_ex_hom 0.0
H2O_ex_hom 23.33333333333333
CO2_ex_carb 20.0
H2_ex_carb 0.0
CO_ex_carb -40.0
H2O_ex_carb -3.333333333333332


# Add Heterotroph

In [8]:
#Bsubt = cobra.io.read_sbml_model(join(data_dir,'Models/iJO1366.xml'))
Geob = cobra.io.read_sbml_model(join(data_dir,'Models/p-thermo_acetone_anaerobic.xml'))

Change objective function and remove constraints for ATP uptake for maintenance. This allows to ignore both GAM and NGAM

In [9]:
Geob.objective = "EX_act_e"

Geob.reactions.ATPM.bounds = 0,1000
Geob.reactions.ATPM.bounds

(0, 1000)

In [21]:
with Geob:
    medium = Geob.medium
    medium["EX_glc__D_e"] = 0
    medium["EX_o2_e"] = 1000
    medium["EX_ac_e"] = Acetate_exc
    Geob.medium = medium
    Geob.optimize()
    Y_PS_carb = Geob.slim_optimize()*3/(Acetate_exc*2)
    Y_PE_carb = -Geob.slim_optimize()*3/CO_exc_carb
    print(Geob.summary(), "\n")
    
with Geob:
    medium = Geob.medium
    medium["EX_glc__D_e"] = 0
    medium["EX_o2_e"] = 1000
    medium["EX_ac_e"] = Acetate_exc
    Geob.medium = medium
    solution = Geob.optimize()
    Y_PS_hom = Geob.slim_optimize()*3/(Acetate_exc*2)
    Y_PE_hom = -Geob.slim_optimize()*3/H2_exc_hom

    print(Geob.summary(), "\n")

Objective
1.0 EX_act_e = 4.2682926829268295

Uptake
------
Metabolite Reaction  Flux  C-Number   C-Flux
      ac_e  EX_ac_e    10         2  100.00%
       h_e   EX_h_e    10         0    0.00%
      o2_e  EX_o2_e 2.927         0    0.00%

Secretion
---------
Metabolite  Reaction   Flux  C-Number  C-Flux
     act_e  EX_act_e -4.268         3  64.02%
     co2_e  EX_co2_e -7.195         1  35.98%
     h2o_e  EX_h2o_e -7.195         0   0.00%
 

Objective
1.0 EX_act_e = 4.2682926829268295

Uptake
------
Metabolite Reaction  Flux  C-Number   C-Flux
      ac_e  EX_ac_e    10         2  100.00%
       h_e   EX_h_e    10         0    0.00%
      o2_e  EX_o2_e 2.927         0    0.00%

Secretion
---------
Metabolite  Reaction   Flux  C-Number  C-Flux
     act_e  EX_act_e -4.268         3  64.02%
     co2_e  EX_co2_e -7.195         1  35.98%
     h2o_e  EX_h2o_e -7.195         0   0.00%
 



In [22]:
print("Carboxydotrophic yield P/e: ", round(Y_PE_carb,3), "[Cmol/emol]\n")
print("Carboxydotrophic yield P/S: ", round(Y_PS_carb,3), "[Cmol/Cmol]\n")
print("Homoacetogenic yield P/e: ", round(Y_PE_hom,3), "[Cmol/emol]\n")
print("Homoacetogenic yield P/S: ", round(Y_PS_hom,3), "[Cmol/Cmol]\n")

Carboxydotrophic yield P/e:  0.32 [Cmol/emol]

Carboxydotrophic yield P/S:  0.64 [Cmol/Cmol]

Homoacetogenic yield P/e:  0.32 [Cmol/emol]

Homoacetogenic yield P/S:  0.64 [Cmol/Cmol]



In [12]:
Builder(model = Geob, map_json = join(data_dir,'Models/EscherMaps/P-thermo_acetone_map.json'), reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'IDPh': 5.7317073170731705, 'CAT': 0.0, 'PDHam1hi': 0.0, 'HYDA': 0.0, 'MALHYDRO': 0.0, …

In [13]:
Geob.metabolites.g3p_c

Metabolite identifier,g3p_c
Name,Glyceraldehyde 3-phosphate
Memory address,0x07f78893ff5f8
Formula,C3H6O6P
Compartment,c
In 14 reaction(s),"DXPS, TRPS1, DRPA, TGBPA, TALA, FBA, TPI, XU5PGT, TRPS3, GAPD, GAPDH, TKT2, TKT1, PYDXS"
